In [2]:
import vtk
from vtk import *

reader = vtkXMLImageDataReader()
reader.SetFileName('Data/Isabel_2D.vti')
reader.Update()
data = reader.GetOutput()
numCells = data.GetNumberOfCells()
dimensions = data.GetDimensions()
numPoints = data.GetNumberOfPoints()
print("cells",numCells)
print("dim",dimensions[0], "x", dimensions[1], "x", dimensions[2])
print("points",numPoints)

cells 62001
dim 250 x 250 x 1
points 62500


In [3]:
pressure_values = data.GetPointData().GetArray('Pressure')
pressure_range = pressure_values.GetRange()
print("Pressure range:",pressure_range[0], pressure_range[1])
total_pressure = 0
pressure_points = 0
for i in range(pressure_values.GetNumberOfTuples()):
    total_pressure += pressure_values.GetTuple1(i)
    pressure_points += 1
average_pressure = total_pressure / pressure_points
print("Mean pressure:",average_pressure)


Pressure range: -1434.8590087890625 630.5694580078125
Mean pressure: 240.77722069091325


In [4]:
sample_cell = data.GetCell(0)

print("Number of points in sample cell:",sample_cell.GetNumberOfPoints())
print("Number of edges in sample cell:",sample_cell.GetNumberOfEdges())
print("Number of faces in sample cell:",sample_cell.GetNumberOfFaces())

cell_vertices = sample_cell.GetPointIds()

print("Vertex indices: ",cell_vertices.GetId(0),cell_vertices.GetId(1), cell_vertices.GetId(3), cell_vertices.GetId(2)) 
x_total = 0
y_total = 0
z_fixed = 25 
print("Coordinates of vertex:")
for i in range(4):
    vertex_coord = data.GetPoint(cell_vertices.GetId(i))
    x_total += vertex_coord[0]
    y_total += vertex_coord[1]
    print(vertex_coord)
x_mean = x_total / 4
y_mean = y_total / 4
print("The 3D coordinates of centroid: ",x_mean,y_mean, z_fixed)
pressure_data = data.GetPointData().GetArray('Pressure')
pressure_total = 0
print("Pressure at vertex:")
for i in range(4):
    vertex_pressure = pressure_data.GetTuple1(cell_vertices.GetId(i))
    print(vertex_pressure)
    pressure_total += vertex_pressure

pressure_average = pressure_total/4
print("Mean pressure:", pressure_average)


Number of points in sample cell: 4
Number of edges in sample cell: 4
Number of faces in sample cell: 0
Vertex indices:  0 1 251 250
Coordinates of vertex:
(0.0, 0.0, 25.0)
(1.0, 0.0, 25.0)
(0.0, 1.0, 25.0)
(1.0, 1.0, 25.0)
The 3D coordinates of centroid:  0.5 0.5 25
Pressure at vertex:
477.527587890625
474.79827880859375
478.0115661621094
467.60699462890625
Mean pressure: 474.4861068725586


In [6]:

    new_poly = vtk.vtkPolyData()
    new_points = vtk.vtkPoints()
    colors = vtk.vtkUnsignedCharArray()
    colors.SetNumberOfComponents(3)
    colors.SetName("Colors")

    for i in range(4):
        vertex_coordinates = data.GetPoint(cell_vertices.GetId(i))
        new_points.InsertNextPoint(vertex_coordinates)
    colors.InsertNextTuple3(0, 0, 255) 
    colors.InsertNextTuple3(255, 0, 0)  
    colors.InsertNextTuple3(0, 255, 0)  
    colors.InsertNextTuple3(255, 25, 0)   
    new_poly.SetPoints(new_points)
    new_poly.GetPointData().SetScalars(colors)
    glyph_filter = vtk.vtkVertexGlyphFilter()
    glyph_filter.SetInputData(new_poly)
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(glyph_filter.GetOutputPort())
    actor = vtk.vtkActor()
    actor.GetProperty().SetPointSize(5)
    actor.SetMapper(mapper)
    renderer = vtk.vtkRenderer()
    renderer.AddActor(actor)
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)
    render_window.Render()
    render_window_interactor.Start()